Первое задание

In [221]:
import pandas as pd
import seaborn as sns
import numpy as np
import io
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.decomposition import PCA

In [222]:
df = pd.read_csv("data.csv", index_col='id')
df[['X']].mean()

X    12.9
dtype: float64

In [223]:
df[['Y']].mean()

Y    34.9
dtype: float64

In [224]:
reg = LinearRegression().fit(df[['X']], df[['Y']])
reg.intercept_

array([2.99323973])

In [225]:
reg.coef_

array([[2.47339227]])

In [226]:
y = reg.predict(df[['X']])
r2 = r2_score(df.Y, y)
r2

0.7792804938661662

Второе задание

In [227]:
df1 = pd.read_csv("fish_train.csv")
df1

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Roach,150.0,20.4,22.0,24.7,5.8045,3.7544
1,Perch,250.0,25.4,27.5,28.9,7.2828,4.5662
2,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340
3,Bream,925.0,36.2,39.5,45.3,18.7542,6.7497
4,Perch,110.0,19.0,21.0,22.5,5.6925,3.5550
...,...,...,...,...,...,...,...
114,Parkki,273.0,23.0,25.0,28.0,11.0880,4.1440
115,Pike,500.0,42.0,45.0,48.0,6.9600,4.8960
116,Whitefish,1000.0,37.3,40.0,43.5,12.3540,6.5250
117,Perch,120.0,20.0,22.0,23.5,6.1100,3.4075


In [228]:
X = df1.drop(columns="Weight")
y = df1['Weight']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13, stratify=df1['Species'])
X_train = X_train.drop('Species', axis=1)
X_test = X_test.drop('Species', axis=1)


In [229]:
X_train['Width'].mean()

4.600478947368421

In [230]:
reg = LinearRegression()
reg.fit(X_train, y_train)
r2_score(y_test, reg.predict(X_test))

0.8110456956218777

In [231]:
pca = PCA(n_components=1, svd_solver='full')
pca.fit(X_train[['Length1', 'Length2', 'Length3']])
print(pca.explained_variance_ratio_)


[0.99631672]


In [232]:
X_train['Lengths'] = pca.fit_transform(X_train[['Length1', 'Length2', 'Length3']])[:, 0]
X_train = X_train[['Height','Lengths','Width']]

In [233]:
X_test['Lengths'] = pca.transform(X_test[['Length1', 'Length2', 'Length3']])[:, 0]
X_test = X_test[['Height','Lengths','Width']]

In [234]:
l_reg = LinearRegression()
l_reg.fit(X_train, y_train)
r2_score(y_test, l_reg.predict(X_test))


0.8177916663297269

In [235]:
X_train[['Width', 'Height', 'Lengths']] = X_train[[ 'Width', 'Height', 'Lengths']].apply(lambda x: x**3)
X_test[['Width', 'Height', 'Lengths']] = X_test[[ 'Width', 'Height', 'Lengths']].apply(lambda x: x**3)
print(X_train['Width'].mean())

135.48357695658208


In [236]:
l_reg1 = LinearRegression()
l_reg1.fit(X_train, y_train)
r2_score(y_test, l_reg1.predict(X_test))

0.9290745197475057

In [239]:
cat_mask = (df1.dtypes.values==object)
X_train1 = pd.get_dummies(df1[df1.columns[cat_mask]], sparse=True, drop_first=True)
X_train_new = pd.concat([X_train, X_train1],axis=1)
X_test1 = pd.get_dummies(df1[df1.columns[cat_mask]], sparse=True, drop_first=True)
X_test_new = pd.concat([X_test, X_test1], axis=1)
X_train_ = X_train_new.dropna()
X_test_ = X_test_new.dropna()

In [240]:
l_reg2 = LinearRegression()
l_reg2.fit(X_train_, y_train)
r2_score(y_test, l_reg2.predict(X_test_))

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


0.9516847741013688